In [1]:
from keras.layers import (Conv3D, BatchNormalization, AveragePooling3D, concatenate, Lambda,
                          Activation, SpatialDropout3D, Input, GlobalAvgPool3D, Dense)
from keras.regularizers import l2 as l2_penalty
from keras.models import Model
from keras.callbacks import ReduceLROnPlateau, TensorBoard, ModelCheckpoint,Callback
import tensorflow as tf

from keras.utils import multi_gpu_model
#from mylib.models.metrics import invasion_acc, invasion_precision, invasion_recall, invasion_fmeasure

PARAMS = {
    'activation': lambda: Activation('relu'),  # the activation functions
    'bn_scale': True,  # whether to use the scale function in BN
    'weight_decay': 0.,  # l2 weight decay
    'kernel_initializer': 'he_uniform',  # initialization
    'first_scale':None,   # lambda x: x / 128. - 1.,  # the first pre-processing function
    'dhw': [32, 32, 32],  # the input shape
    'k': 16,  # the `growth rate` in DenseNet
    'bottleneck': 2,  # the `bottleneck` in DenseNet
    'compression': 2,  # the `compression` in DenseNet
    'first_layer': 32,  # the channel of the first layer
    'down_structure': [4, 4],  # the down-sample structure
    'output_size': 2  # the output number of the classification head
}


def _conv_block(x, filters):
    bn_scale = PARAMS['bn_scale']
    activation = PARAMS['activation']
    kernel_initializer = PARAMS['kernel_initializer']
    weight_decay = PARAMS['weight_decay']
    bottleneck = PARAMS['bottleneck']

    x = BatchNormalization(scale=bn_scale, axis=-1)(x)
    x = activation()(x)
    x = Conv3D(filters * bottleneck, kernel_size=(1, 1, 1), padding='same', use_bias=False,
               kernel_initializer=kernel_initializer,
               kernel_regularizer=l2_penalty(weight_decay))(x)
    #x = SpatialDropout3D(0.1)(x)
    x = BatchNormalization(scale=bn_scale, axis=-1)(x)
    x = activation()(x)
    x = Conv3D(filters, kernel_size=(3, 3, 3), padding='same', use_bias=True,
               kernel_initializer=kernel_initializer,
               kernel_regularizer=l2_penalty(weight_decay))(x)
    return x


def _dense_block(x, n):
    k = PARAMS['k']

    for _ in range(n):
        conv = _conv_block(x, k)
        x = concatenate([conv, x], axis=-1)
    return x


def _transmit_block(x, is_last):
    bn_scale = PARAMS['bn_scale']
    activation = PARAMS['activation']
    kernel_initializer = PARAMS['kernel_initializer']
    weight_decay = PARAMS['weight_decay']
    compression = PARAMS['compression']

    x = BatchNormalization(scale=bn_scale, axis=-1)(x)
    x = activation()(x)
    if is_last:
        x = GlobalAvgPool3D()(x)
    else:
        *_, f = x.get_shape().as_list()
        x = Conv3D(f // compression, kernel_size=(1, 1, 1), padding='same', use_bias=True,
                   kernel_initializer=kernel_initializer,
                   kernel_regularizer=l2_penalty(weight_decay))(x)
        x = AveragePooling3D((2, 2, 2), padding='valid',)(x)
    return x


def get_model(weights=None, **kwargs):
    for k, v in kwargs.items():
        assert k in PARAMS
        PARAMS[k] = v
    print("Model hyper-parameters:", PARAMS)

    dhw = PARAMS['dhw']
    first_scale = PARAMS['first_scale']
    first_layer = PARAMS['first_layer']
    kernel_initializer = PARAMS['kernel_initializer']
    weight_decay = PARAMS['weight_decay']
    down_structure = PARAMS['down_structure']
    output_size = PARAMS['output_size']

    shape = dhw + [1]

    inputs = Input(shape=shape)

    if first_scale is not None:
        scaled = Lambda(first_scale)(inputs)
    else:
        scaled = inputs
    conv = Conv3D(first_layer, kernel_size=(3, 3, 3), padding='same', use_bias=True,
                  kernel_initializer=kernel_initializer,
                  kernel_regularizer=l2_penalty(weight_decay))(scaled)

    downsample_times = len(down_structure)
    for l, n in enumerate(down_structure):
        db = _dense_block(conv, n)
        conv = _transmit_block(db, l == downsample_times - 1)

    if output_size == 1:
        last_activation = 'sigmoid'
    else:
        last_activation = 'softmax'

    outputs = Dense(output_size, activation=last_activation,
                    kernel_regularizer=l2_penalty(weight_decay),
                    kernel_initializer=kernel_initializer)(conv)

    model = Model(inputs, outputs)
    model.summary()

    if weights is not None:
        model.load_weights(weights, by_name=True)
    return model





Using TensorFlow backend.


In [2]:
import numpy as np
import collections
from itertools import repeat
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split

def rotation(array, angle):
    '''using Euler angles method.
    @author: renchao
    @params:
        angle: 0: no rotation, 1: rotate 90 deg, 2: rotate 180 deg, 3: rotate 270 deg
    '''
    #
    X = np.rot90(array, angle[0], axes=(0, 1))  # rotate in X-axis
    Y = np.rot90(X, angle[1], axes=(0, 2))  # rotate in Y'-axis
    Z = np.rot90(Y, angle[2], axes=(1, 2))  # rotate in Z"-axis
    return Z


def reflection(array, axis):
    '''
    @author: renchao
    @params:
        axis: -1: no flip, 0: Z-axis, 1: Y-axis, 2: X-axis
    '''
    if axis != -1:
        ref = np.flip(array, axis)
    else:
        ref = np.copy(array)
    return ref


def crop(array, zyx, dhw):
    z, y, x = zyx
    d, h, w = dhw
    cropped = array[z - d // 2:z + d // 2,
              y - h // 2:y + h // 2,
              x - w // 2:x + w // 2]
    return cropped


def random_center(shape, move):
    offset = np.random.randint(-move, move + 1, size=3)
    zyx = np.array(shape) // 2 + offset
    return zyx


def _ntuple(n):
    def parse(x):
        if isinstance(x, collections.Iterable):
            return x
        return tuple(repeat(x, n))

    return parse


_single = _ntuple(1)
_pair = _ntuple(2)
_triple = _ntuple(3)
_quadruple = _ntuple(4)

class Transform:
    '''The online data augmentation, including:
    1) random move the center by `move`
    2) rotation 90 degrees increments
    3) reflection in any axis
    '''

    def __init__(self, size, move):
        self.size = _triple(size)
        self.move = move

    def __call__(self, arr, aux=None):
        shape = arr.shape
        if self.move is not None:
            center = random_center(shape, self.move)
            arr_ret = crop(arr, center, self.size)
            angle = np.random.randint(4, size=3)
            arr_ret = rotation(arr_ret, angle=angle)
            axis = np.random.randint(4) - 1
            arr_ret = reflection(arr_ret, axis=axis)
            arr_ret = np.expand_dims(arr_ret, axis=0)
            if aux is not None:
                aux_ret = crop(aux, center, self.size)
                aux_ret = rotation(aux_ret, angle=angle)
                aux_ret = reflection(aux_ret, axis=axis)
                aux_ret = np.expand_dims(aux_ret, axis=0)
                return arr_ret, aux_ret
            return arr_ret
        else:
            center = np.array(shape) // 2
            arr_ret = crop(arr, center, self.size)
            arr_ret = np.expand_dims(arr_ret, axis=0)
            if aux is not None:
                aux_ret = crop(aux, center, self.size)
                aux_ret = np.expand_dims(aux_ret, axis=0)
                return arr_ret, aux_ret
            return arr_ret
        
class EarlyStoppingByLossVal(Callback):
    def __init__(self, monitor1, monitor2,value,verbose):
        super(Callback,self).__init__()
        self.monitor1 = monitor1
        self.monitor2 = monitor2
        
        self.value = value
        self.verbose = verbose
        
    def on_epoch_end(self,epoch,logs={}):
        current=logs.get(self.monitor1)**0.5+logs.get(self.monitor2)**2
        
        if current is None:
            warnings.warn("Early stopping warning",RuntimeWarning)
        if current < self.value:
            if self.verbose>0:
                print("Epoch %05d: early stopping THR" % epoch)
            self.model.stop_training = True
                

In [6]:

import pandas as pd
import numpy as np
import keras,os,csv
from tqdm import tqdm
from scipy.ndimage import zoom
from keras.callbacks import ReduceLROnPlateau, TensorBoard, ModelCheckpoint,Callback

# 读取训练数据
voxel_train = []   #用于存储训练数据的voxel
seg_train = []     #用于存储训练数据的seg

for i in tqdm(range(584), desc='reading train_data'):           #写入训练数据进度
    try:
        tmp = np.load('train_val/candidate{}.npz'.format(i))    #依次读取训练数据中的candidate{i}文件
    except FileNotFoundError:                                   #无该文件时直接进入下一次循环
        continue
    try:
        voxel_train = np.append(voxel_train, np.expand_dims(tmp['voxel'], axis=0), axis=0)  #向voxel_train中添加读取的voxel向量
        seg_train = np.append(seg_train, np.expand_dims(tmp['seg'], axis=0), axis=0)        #向seg_train中添加读取的seg向量
    except ValueError:
        voxel_train = np.expand_dims(tmp['voxel'], axis=0)      #写入初次读取的voxel
        seg_train = np.expand_dims(tmp['seg'], axis=0)          #写入初次读取的seg

#读取训练集的标签
train_label = pd.read_csv('train_val.csv').values[:, 1].astype(int)
#print(train_label.shape)
train_label = np.concatenate((train_label,train_label),axis=0)
#print(train_label.shape)

seg_train = seg_train.astype(np.int)         #将seg的布尔array转换为1/0整数
X_train = voxel_train*seg_train              #抠取结节

X_train=X_train.astype(np.float32)
X_train/=128.-1.

training_batch_size = X_train.shape[0]       #训练数据集的数量
#print(X_train.shape)

X_train_new=crop(X_train[0],(50,50,50),(32,32,32))

#print(X_train_new.shape) 
X_train_new=np.expand_dims(X_train_new,axis=0)
#print(X_train_new.shape) 

for i in tqdm(range(training_batch_size-1),desc='croping'):    #将数据大小截取为(32,32,32)
    X_train_new=np.append(X_train_new,np.expand_dims(crop(X_train[i+1],(50,50,50),(32,32,32)),axis=0),axis=0)
#print(X_train_new.shape)   

for i in tqdm(range(training_batch_size),desc='transforming'): #对数据做transform处理
    X_train_new=np.append(X_train_new,Transform(32,5)(X_train[i]),axis=0)
#print(X_train_new.shape) 
weight = np.random.beta(0.2, 0.2, training_batch_size*2)
X_weight = weight.reshape(training_batch_size*2,1,1,1)
for i in tqdm(range(training_batch_size*2),desc='mixuping'):   #对数据做mixup处理
    index = np.random.randint(1,training_batch_size*2)
    X_train_new=np.append(X_train_new,np.expand_dims(X_weight[i]*X_train_new[i]+(1-weight[i])*X_train_new[index],axis=0),axis=0)
    train_label = np.append(train_label, np.expand_dims(weight[i]*train_label[i]+(1-weight[i])*train_label[index],axis=0),axis=0)
print(X_train_new.shape) 

del X_train
X_train_new = X_train_new.reshape(X_train_new.shape[0], 32, 32, 32, 1) 
#print(X_train_new.shape)
train_label = keras.utils.to_categorical(train_label, 2)
print(train_label.shape)

for root, dirs,files in os.walk('test'):   
    filename = files

def sort_key(s):
    return int(s[9:-4])

test_number = sorted(filename,key = sort_key)         #获得排序的测试集

#读取测试数据
voxel_test = []     #用于存储测试数据的voxel
seg_test = []       #用于存储测试数据的seg

for i in tqdm(range(584), desc='reading test_data'):    #写入测试数据的进度
    try:
        tmp = np.load('test/candidate{}.npz'.format(i)) #依次读取测试数据中的candidate{i}文件
    except FileNotFoundError:                           #无该文件时直接进入下一次循环
        continue
    try:
        voxel_test = np.append(voxel_test, np.expand_dims(tmp['voxel'], axis=0), axis=0)    #向voxel_test中添加读取的voxel向量
        seg_test = np.append(seg_test, np.expand_dims(tmp['seg'], axis=0), axis=0)          #向seg_test中添加读取的seg向量
    except ValueError:
        voxel_test = np.expand_dims(tmp['voxel'], axis=0)   #写入初次读取的voxel
        seg_test = np.expand_dims(tmp['seg'], axis=0)       #写入初次读取的seg

seg_test = seg_test.astype(np.int)      #将seg布尔array转换为1/0整数
X_test= voxel_test*seg_test             #抠取结节

X_test=X_test.astype(np.float32)
X_test/=128.-1.

test_batch_size = X_test.shape[0]  #测试数据集的数量
#print(X_test.shape)

X_test_new=crop(X_test[0],(50,50,50),(32,32,32))

X_test_new=np.expand_dims(X_test_new,axis=0)
#print(X_test_new.shape) 

for i in tqdm(range(test_batch_size-1),desc='croping'):
    X_test_new=np.append(X_test_new,np.expand_dims(crop(X_test[i+1],(50,50,50),(32,32,32)),axis=0),axis=0)
#print(X_test_new.shape)   
del X_test
X_test_new = X_test_new.reshape(X_test_new.shape[0], 32, 32, 32, 1)     #将测试数据集整合成5d张量
print(X_test_new.shape)


croping: 100%|██████████| 116/116 [00:00<00:00, 794.82it/s]


In [4]:
#训练模型
if __name__ == '__main__':
    
    
   # with tf.device('/cpu:3'):
    X_train_new,x_val, train_label, y_val =train_test_split(X_train_new,train_label,test_size=160,shuffle=True)
    model = get_model(weights=None)
    #parallel_model = multi_gpu_model(model, gpus=2)
    
    model.compile(loss='categorical_crossentropy', optimizer='adamax',
                  metrics=["accuracy"])
    
    filepath="model_weight.h5"
    reduce_lr = ReduceLROnPlateau(monitor='acc', factor=0.2,patience=3, min_lr=0.0005)
    checkpoint = ModelCheckpoint(filepath,monitor='val_loss',verbose=0,save_best_only=True,mode='min',period=1)
    callbacks_list=[reduce_lr,
                    checkpoint,
                   ]
    model.fit(X_train_new,train_label,batch_size=30,epochs=26,validation_data=(x_val,y_val),callbacks=callbacks_list,shuffle=True)
    #保存模型
    model.save('model.h5')

Model hyper-parameters: {'activation': <function <lambda> at 0x7ff488a4c730>, 'bn_scale': True, 'weight_decay': 0.0, 'kernel_initializer': 'he_uniform', 'first_scale': None, 'dhw': [32, 32, 32], 'k': 16, 'bottleneck': 2, 'compression': 2, 'first_layer': 32, 'down_structure': [4, 4], 'output_size': 2}
Instructions for updating:
Colocations handled automatically by placer.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 32, 1 0                                            
__________________________________________________________________________________________________
conv3d_1 (Conv3D)               (None, 32, 32, 32, 3 896         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor 

Train on 1700 samples, validate on 160 samples
Epoch 1/26
1700/1700 [==============================] - 64s 38ms/step - loss: 0.6646 - acc: 0.6171 - val_loss: 0.6996 - val_acc: 0.6875
Epoch 2/26
1700/1700 [==============================] - 57s 33ms/step - loss: 0.6521 - acc: 0.6141 - val_loss: 0.6275 - val_acc: 0.6813
Epoch 3/26
1700/1700 [==============================] - 57s 33ms/step - loss: 0.6392 - acc: 0.6306 - val_loss: 0.8515 - val_acc: 0.5813
Epoch 4/26
1700/1700 [==============================] - 57s 33ms/step - loss: 0.6428 - acc: 0.6188 - val_loss: 0.6487 - val_acc: 0.6750
Epoch 5/26
1700/1700 [==============================] - 57s 33ms/step - loss: 0.6335 - acc: 0.6341 - val_loss: 0.6444 - val_acc: 0.6688
Epoch 6/26
1700/1700 [==============================] - 57s 33ms/step - loss: 0.6286 - acc: 0.6435 - val_loss: 0.7313 - val_acc: 0.4938
Epoch 7/26
1700/1700 [==============================] - 57s 33ms/step - loss: 0.6281 - acc: 0.6353 - val_loss: 1.7523 - val_acc: 0.4438
E

In [10]:
#预测并存储结果
import numpy  
y_pred=model.predict(X_test_new)

test_label = []
test_label.append(['id','Predicted'])
for i in range(test_batch_size):
    test_label.append([test_number[i][:-4],y_pred[i][1]])
    
with open('Submission.csv', 'w',newline='') as f:
    writer = csv.writer(f)
    writer.writerows(test_label)